# Bag Of Visual Words Approach

In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import pylab as pl
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import svm
import pandas as pd

# Loading the Images

In [2]:
def load_data():
    bdir = './Content/Bikes/'
    hdir = './Content/Horses/'
    
    img_paths_horse = []
    img_paths_bike = []
    img_paths = []

    for i in os.listdir(hdir):
        img_paths_horse.append(os.path.join(hdir, i))

    for i in os.listdir(bdir):
        img_paths_bike.append(os.path.join(bdir, i))

    img_paths = img_paths_bike + img_paths_horse

    img_class0 = [0]*len(img_paths_bike)
    img_class1 = [1]*len(img_paths_horse)

    img_class = img_class0 + img_class1

    return img_paths, img_class
    

# Splitting the data into train and test sets

In [3]:
def train_test(img_paths, img_class):
    random.seed(23)
    train_imgs_paths, test_imgs_paths, train_labels, test_labels = train_test_split(img_paths, img_class, train_size=0.8, random_state=42,shuffle = True, stratify = img_class)

    return train_imgs_paths, test_imgs_paths, train_labels, test_labels

# Resizing the images

In [4]:
def resize_imgs(l):
    imgs = []

    for i in l:
        img = cv2.imread(i)
        img = cv2.resize(img,(275,180))
        imgs.append(img)

    imgs = np.asarray(imgs)
    return imgs

# Extracting descriptors of the images

In [5]:
def get_descriptors(imgs):
    extractor = cv2.SIFT_create()
    descriptors = np.asarray([])

    for img in imgs:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        kps, descs = extractor.detectAndCompute(gray, None)
        if type(descs) == np.ndarray :
            if descriptors.shape[0] == 0:
                descriptors = descs
            else:
                descriptors = np.concatenate((descriptors, descs), axis=0)

    return descriptors

# Getting the Vocabulary using K-Means Clustering

In [6]:
def get_words(descriptors):
    kmeans = KMeans(n_clusters = 512)
    kmeans.fit(descriptors)

    return kmeans

# Constructing Histograms/Bag Of Keypoints

In [7]:
def build_histograms(imgs, words, nClusters):
    histograms = []
    extractor = cv2.SIFT_create()
    for img in imgs:
        histogram = [0]*nClusters
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        kps, descs = extractor.detectAndCompute(gray, None)

        if(type(descs) == np.ndarray):
            prediction = words.predict(descs)
            for i in prediction:
                histogram[i] = histogram[i] + 1
        
        histograms.append(histogram)

    return np.asarray(histograms)

# Models

# 1. SVM

In [8]:
def train(features, train_labels):
    model = svm.SVC(C = 0.005, kernel = 'linear')
    model.fit(features, train_labels)

    return model

In [9]:
img_paths, img_class = load_data()
train_imgs_paths, test_imgs_paths, train_labels, test_labels = train_test(img_paths, img_class)

train_imgs = resize_imgs(train_imgs_paths)
descriptors = get_descriptors(train_imgs)
words = get_words(descriptors)
features = build_histograms(train_imgs, words, nClusters = 512)

model = train(features, train_labels)

In [10]:
test_imgs = resize_imgs(test_imgs_paths)
test_features = build_histograms(test_imgs, words, nClusters = 512)
accuracy = model.score(test_features, test_labels)
accuracy

0.9722222222222222

# 2. Logistic Regression

In [39]:
def train1(features, train_labels):
    model1 = LogisticRegression(max_iter = 1000)
    model1.fit(features, train_labels)

    return model1

In [40]:
model1 = train1(features, train_labels)

accuracy1 = model1.score(test_features, test_labels)
accuracy1

0.9722222222222222

# 3. K-Nearest Neighbor

In [41]:
def train2(features, train_labels):
    model2 = KNeighborsClassifier(n_neighbors = 5)
    model2.fit(features, train_labels)

    return model2

In [42]:
model2 = train2(features, train_labels)

accuracy2 = model2.score(test_features, test_labels)
accuracy2

0.9166666666666666